В ходе данного соревнования, мы перепробовали большое количество алгоритмов ML, также вариантов извлечения фич.
Финальный алгоритм выглядит таким образом:

### Feature extraction
Для извлечения фич использовалась библиотека Surfboard[2], предоставляющая высокоуровневый API для извлечения фич и статистики по ним.

**Извлекались следующие фичи**:

Основная частота (f0), логарифм энергии (logE), спектральный центроид (spectral centroid),
форманты (formants) (1 - 4), громкость, jitters, shimmers, пересечение нуля (zerocrossing) и MFCC (первые 16)

**Используя представленные статистики:**

Среднее, СКО, минимум, максимум, коэффициент асимметрии (skewness),коэффициент островершинности (kurtosis),
первая производная среднего, первая производная СКО, межквартальное размах.

Такой набор фич составлен на основе проченных научных трудов [1,3], а также из общих соображений.

Была попытка извлекать не статистики фич, а сами фичи, но это не принесло улучшения результатов,
поскольку большинство фич было откинуто на этапе Feature selection

В конечном итоге названия столбцов датасета имеют следующий вид: фича_статистика(_№) (номер только для MFCC)

In [1]:
# Вот так выглядил датасет до Feature selection
import pandas as pd
df = pd.read_csv("features_and_stat_full_2.csv")
df.head(3)

,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,f0_contour_first_derivative_mean,f0_contour_first_derivative_std,f0_contour_q3_q1_range,log_energy,...,mfcc_q3_q1_range_8,mfcc_q3_q1_range_9,mfcc_q3_q1_range_10,mfcc_q3_q1_range_11,mfcc_q3_q1_range_12,mfcc_q3_q1_range_13,mfcc_q3_q1_range_14,mfcc_q3_q1_range_15,mfcc_q3_q1_range_16,target
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,1.570260e-16,27.951188,0.0,-50.789417,...,6.817384,10.575198,5.019678,18.639672,4.687609,3.798230,6.998048,4.952859,7.033751,1
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-1.652425e-16,29.417767,0.0,-52.897926,...,6.969563,17.334488,9.290317,6.188273,5.173421,7.520164,4.676119,8.310646,5.102108,1
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,0.000000e+00,19.592358,0.0,-57.779049,...,12.634435,14.232127,10.144253,10.668671,12.063463,9.836006,11.097036,10.172810,9.805441,1


### Feature selection
Для улучшения скорости обучения и точности классификаторов использовался выбор фич на основе взаимной информации
(Mutual information). Были выбраны 120 столбцов с наивысшими показателями.

### Классификатор
После многочисленных экспериментов было решено использовать стэкинг из 4-ех классификаторов
 * MLP (размер  скрытого слоя - 200)
 * Случайный лес (500 деревьев)
 * Логистическая регрессия (параметр регуляризации - l2, C = 0.01)
 * SVC (С = 0,01)

Перед тем как "скормить" классификатору фичи, они были стандартизированы.
Эти 4 классификатора показали наилучшие результаты на кросс-валидации, поэтому было решено объединить их в стэк.
Также они рекомендовались в ряде научных работ [1,3]
В итоге, ансамбль имел точность ~ 0.82 на кросс валидации по 5 блокам.

### Возможные улучшения
Матрица ошибок показывает, что все классификаторы плохо различают 4 и 2 класс.
Если убрать один из них, точность ансамбля приближается к ~0.95. К сожалению, за время соревнования нам так
и не удалось найти такие фичи и/или классификаторы, которые смогли бы решить данную проблему

### Источники
 1. Lijiang Chen et al. , Speech emotion recognition: Features and classification models,
 DOI: https://doi.org/10.1016/j.dsp.2012.05.007
 2. Raphael Lenain et al. , Surfboard: Audio Feature Extraction for Modern Machine Learning,
 Ref: arXiv:2005.08848
 3. Casale et al. Speech Emotion Classification Using Machine Learning Algorithms.
 DOI: 10.1109/ICSC.2008.43